In [50]:
import csv
import re

In [3]:
import requests
from bs4 import BeautifulSoup

/Users/facade/projects/irasutoya/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [25]:
USER_AGENT_CHROME = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"

## 1. shousai_categories

In [17]:
resp = requests.get("https://www.irasutoya.com/")
doc = BeautifulSoup(resp.content.decode('utf-8'), 'html.parser')

In [18]:
cont = doc.select_one("#sidebar-wrapper #sidebar #Label1")
if getattr(cont.select_one(":scope > h2"), 'text') != "詳細カテゴリー":
    raise ValueError("詳細カテゴリー header not found")

shousai_cates = []
for i, e in enumerate(cont.select(":scope > .widget-content.list-label-widget-content > ul > li")):
    a = e.select_one(":scope > a")
    if not a or a.attrs.get('dir') != 'ltr':
        raise ValueError("Unexpected list item: '{}'".format(e))
    url = a['href']
    scate = a.text

    shousai_cates.append({
        'url': url,
        'name': scate,
    })
    print("{:02d} {}: {}".format(i+1, scate, url))
    

01 リクエスト: https://www.irasutoya.com/search/label/%E3%83%AA%E3%82%AF%E3%82%A8%E3%82%B9%E3%83%88
02 こども: https://www.irasutoya.com/search/label/%E3%81%93%E3%81%A9%E3%82%82
03 職業: https://www.irasutoya.com/search/label/%E8%81%B7%E6%A5%AD
04 病気: https://www.irasutoya.com/search/label/%E7%97%85%E6%B0%97
05 ポーズ: https://www.irasutoya.com/search/label/%E3%83%9D%E3%83%BC%E3%82%BA
06 会社: https://www.irasutoya.com/search/label/%E4%BC%9A%E7%A4%BE
07 お金: https://www.irasutoya.com/search/label/%E3%81%8A%E9%87%91
08 道具: https://www.irasutoya.com/search/label/%E9%81%93%E5%85%B7
09 ビジネス: https://www.irasutoya.com/search/label/%E3%83%93%E3%82%B8%E3%83%8D%E3%82%B9
10 学校: https://www.irasutoya.com/search/label/%E5%AD%A6%E6%A0%A1
11 ファッション: https://www.irasutoya.com/search/label/%E3%83%95%E3%82%A1%E3%83%83%E3%82%B7%E3%83%A7%E3%83%B3
12 医療: https://www.irasutoya.com/search/label/%E5%8C%BB%E7%99%82
13 事故: https://www.irasutoya.com/search/label/%E4%BA%8B%E6%95%85
14 違反: https://www.irasutoya.com/search/label

## 2. irasuto_lists

In [86]:
resp = requests.get("https://www.irasutoya.com/search/label/%E5%8D%97%E5%9B%BD",
                   headers={'User-Agent': USER_AGENT_CHROME,
                           'Accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8"}) # 南国
doc = BeautifulSoup(resp.content.decode('utf-8'), 'html.parser')

In [87]:
#contents = doc.select("#main #Blog1 .blog-posts.hfeed .date-outer")  # -> this results in fewer results?
contents = doc.select("#post")
print("#content=", len(contents))
for ic, c in enumerate(contents):
    #print(c.select_one("#post > .boxmeta"))
    #t = getattr(c.select_one("#post > .boxmeta > h2 > a"), 'text', "")
    a = c.select_one(".boxmeta > h2 > a")
    t = getattr(a, 'text', "")
    url = a['href']
    # grab thumbnail (small size)
    thumb_url = None
    boxim = str(c.select_one(".boxim"))
    m = re.search(r"document\.write\(bp_thumbnail_resize\(\"(.+)\",", boxim)
    if m:
        thumb_url = m.group(1)
    
    print("{:01d} {} ({} thumb={})".format(ic+1, t, url, thumb_url))

nav_e = doc.select_one("#navigation #blog-pager")
next_pg_es = nav_e.select("#blog-pager-older-link a.blog-pager-older-link")
if len(next_pg_es) == 0:
    home_e = nav_e.select_one("#home-link .home-link .navibtn")
    assert home_e is not None
else:
    assert len(next_pg_es) in [1, 2]
    for next_pg_e in next_pg_es:
        im_e = next_pg_e.select_one(":scope > img[alt]")
        print(next_pg_e)
        if im_e['alt'] == "次のページ":
            print("Next page url:", next_pg_e['href'])

#content= 20
1 フランキーのイラスト（ONE PIECE） (https://www.irasutoya.com/2021/01/franky.html thumb=https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEibyeQ970m83yM43EI4t_wfvMdgoP_wqk6pkG8OiwxUJxxqVqPxq8o1pcQaphSepqt2oM1P5cmxTAlhpJRmt3_0JuYKNggM5qIdNFQwzp0RP8xqEzglhuYTIrG-9aBB-fm2iXT_Qejf5rUJ/s72-c/onepiece08_franky.png)
2 沖縄の獅子舞のイラスト (https://www.irasutoya.com/2020/08/blog-post_82.html thumb=https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEjPQcZhtdpATCFpsEbaTPLmjmqWgsTC119dqK6jttF5nsEkRIOOD13GdoxkxM6XQboJqRoEQ9ZDVTiktVBa9Cxv-YhfS1Y4a1NZH1BL4rG96eX5Gy0qW4ag_mcYMp65zMtFouH_FmB-ibVN/s72-c/okinawa_shishimai.png)
3 半分に割れたココナッツのイラスト (https://www.irasutoya.com/2020/06/blog-post_31.html thumb=https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEhoVm07WHBlDyQdQR-RJDPzZQtEin3_O8_Rq_fr-zpJWdCR-iTwLs2wzfyyXLEz9cBPEa2VnwzRBfnUug3LgZnKZrEJzS54C_SwN255VI1vAbBNB6d3Abev18P5bESkdGYp0KRtlCPq7t__/s72-c/fruit_coconut_half.png)
4 ココナッツのイラスト (https://www.irasutoya.com/2020/06/blog-post_424.htm

In [81]:
next_pg_es[0]

KeyError: 0

## 3. irasuto

In [55]:
resp = requests.get("https://www.irasutoya.com/2024/04/blog-post_16.html")
doc = BeautifulSoup(resp.content.decode('utf-8'), 'html.parser')

In [71]:
d = {}
main_cont = doc.select_one("#main #Blog1 #post")
d['title'] = main_cont.select_one(".title").text
im_e = main_cont.select_one(".entry .separator a")
d['im_url'] = im_e['href']
d['im_thumb_url'] = im_e.select_one('img')['src']

sep_es = doc.select("#main #Blog1 #post > .entry .separator")
if len(sep_es) != 2:
    raise AssertionError("Failed to find img description")
d['description'] = sep_es[1].text

d['tags'] = []
for tag_e in main_cont.select(".titlemeta .category [rel=tag]"):
    d['tags'].append(tag_e.text)

d

{'title': '\n\nギャルピースのイラスト\n\n',
 'im_url': 'https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEg48GxlSXF_4b4XZmtOALPhe3mD5iREyN-Ks6Q2hdviWeDHOcG_AUOS3nn2i-E9g5jD1_7-2o9PZF5MUQEanceM7b07viAr9M6h4C7jDqGhKdF0LzHzn2IBS_A2Fvpv605wIRf9ohIPiv-HStNDjk8JdN2hU-0GTI-OsjRraMo1HnGkTALf6v7qBbHufj04/s776/pose_galpeace_schoolgirl.png',
 'im_thumb_url': 'https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEg48GxlSXF_4b4XZmtOALPhe3mD5iREyN-Ks6Q2hdviWeDHOcG_AUOS3nn2i-E9g5jD1_7-2o9PZF5MUQEanceM7b07viAr9M6h4C7jDqGhKdF0LzHzn2IBS_A2Fvpv605wIRf9ohIPiv-HStNDjk8JdN2hU-0GTI-OsjRraMo1HnGkTALf6v7qBbHufj04/s400/pose_galpeace_schoolgirl.png',
 'description': 'ピースポーズを反対側にして前に突き出すポーズ、ギャルピース（ギャルピ）をする女の子のイラストです。',
 'tags': ['ポーズ', '学校']}

In [67]:
doc.select("#main #Blog1 #post > .entry > .separator")

[]